In [1]:
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
import os
from scipy.spatial import distance
import matplotlib.pyplot as plt
from skimage import feature

In [3]:
dir_object='Objects'

In [4]:
def normalize_feature_vector(feature_vector):
    l2_norm = np.linalg.norm(feature_vector)
    normalized_vector = feature_vector / l2_norm if l2_norm != 0 else feature_vector
    return normalized_vector


In [5]:
def compute_hog_train(img):
    img =cv2.resize(img, (64, 128))
    (hog, hog_image) = feature.hog(img, orientations=9,pixels_per_cell=(8, 8), cells_per_block=(2, 2),block_norm='L2-Hys', visualize=True, transform_sqrt=True)  
    return hog

In [6]:
featurevector=[]
path=[]
for objects in os.listdir(dir_object):
    dir_photos=os.path.join(dir_object,objects,"photos")
    for photos in os.listdir(dir_photos):
        dir_images=os.path.join(dir_photos,photos)
        image=cv2.imread(dir_images,0)
        fv=compute_hog_train(image)
        fv=normalize_feature_vector(fv)
        featurevector.append(fv)
        path.append(dir_images)

In [7]:
gray=cv2.imread("Objects/bicycle/sketches/1681.png",0)

In [8]:
fv = compute_hog_train(gray)
# Normalize the feature vector of the drawn sketch
normalized_sketch_vector = normalize_feature_vector(fv)

# Calculate similarities between the sketch vector and image feature vectors
similarities = {}
for image_path, image_vector in zip(path, featurevector):
    similarity = distance.cosine(normalized_sketch_vector, image_vector)
    if similarity in similarities:
        similarities[similarity].append(image_path)
    else:
        similarities[similarity] = [image_path]

# Sort the similarities in descending order
sorted_similarities = sorted(similarities.keys(), reverse=True)

# Retrieve the ranked images based on the sorted similarities
ranked_images = []
for similarity in sorted_similarities:
    ranked_images.extend(similarities[similarity])
top_k = 10  # Number of top images to display
images = []
num_images = min(len(ranked_images[:top_k]), 10)  # Display up to 10 images
grid_rows = int(np.sqrt(num_images))
grid_cols = int(np.ceil(num_images / grid_rows))
grid_size = 200

grid = np.zeros((grid_rows * grid_size, grid_cols * grid_size, 3), dtype=np.uint8)

for i in range(num_images):
    image = cv2.imread(ranked_images[i])
    image = cv2.resize(image, (grid_size, grid_size))
    row = i // grid_cols
    col = i % grid_cols
    grid[row * grid_size: (row + 1) * grid_size, col * grid_size: (col + 1) * grid_size] = image

cv2.imshow("Images", grid)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [16]:

# Create a black canvas
canvas = np.zeros((1000, 1000, 3), dtype=np.uint8)

# Create windows to display the canvas and the edges
cv2.namedWindow('Canvas')

# Define the mouse callback function
def draw(event, x, y, flags, param):
    global last_x, last_y
    if event == cv2.EVENT_LBUTTONDOWN:
        last_x, last_y = x, y
    elif event == cv2.EVENT_MOUSEMOVE and flags == cv2.EVENT_FLAG_LBUTTON:
        cv2.line(canvas, (last_x, last_y), (x, y), (255, 255, 255), 2)
        last_x, last_y = x, y

cv2.setMouseCallback('Canvas', draw)

# Main loop
while True:
    # Convert the canvas to grayscale
    gray = cv2.cvtColor(canvas, cv2.COLOR_BGR2GRAY)

    # Display the canvas and the edges
    cv2.imshow('Canvas', gray)

    # Wait for a key press
    key = cv2.waitKey(1) & 0xFF

    
    # Check if the canvas is still open before accessing it
    if cv2.getWindowProperty('Canvas', cv2.WND_PROP_VISIBLE) < 1:
        break
    
    if key == ord('s'):
        # Calculate feature vector
        fv = compute_hog_train(gray)
        # Normalize the feature vector of the drawn sketch
        normalized_sketch_vector = normalize_feature_vector(fv)

        # Calculate similarities between the sketch vector and image feature vectors
        similarities = {}
        for image_path, image_vector in zip(path, featurevector):
            similarity = distance.cosine(normalized_sketch_vector, image_vector)
            if similarity in similarities:
                similarities[similarity].append(image_path)
            else:
                similarities[similarity] = [image_path]

        # Sort the similarities in descending order
        sorted_similarities = sorted(similarities.keys(), reverse=True)

        # Retrieve the ranked images based on the sorted similarities
        ranked_images = []
        for similarity in sorted_similarities:
            ranked_images.extend(similarities[similarity])
        top_k = 10  # Number of top images to display
        images = []
        num_images = min(len(ranked_images[:top_k]), 10)  # Display up to 10 images
        grid_rows = int(np.sqrt(num_images))
        grid_cols = int(np.ceil(num_images / grid_rows))
        grid_size = 200

        grid = np.zeros((grid_rows * grid_size, grid_cols * grid_size, 3), dtype=np.uint8)

        for i in range(num_images):
            image = cv2.imread(ranked_images[i])
            image = cv2.resize(image, (grid_size, grid_size))
            row = i // grid_cols
            col = i % grid_cols
            grid[row * grid_size: (row + 1) * grid_size, col * grid_size: (col + 1) * grid_size] = image

        cv2.imshow("Images", grid)
    # If the 'c' key is pressed, clear the canvas
    if key == ord('c'):
        canvas = np.zeros((1000, 1000, 3), dtype=np.uint8)

    # If the 'q' key is pressed, exit the program
    elif key == ord('q'):
        cv2.destroyAllWindows()
        break